In [ ]:
"""
Created on Wed Nov 22 17:27 2023

Apply weights to future as well

Author: @claraburgard

"""

In [1]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-burgardc'


READ IN DATA

In [4]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'


In [5]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [6]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [7]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'quadratic_mixed_mean','quadratic_mixed_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNsmall']

CALVING

In [8]:
inputpath_davison = home_path+'/DATA/SUMMER_PAPER/raw/'
davison_file = xr.open_dataset(inputpath_davison + 'steadystate_davison23.nc')
calving_flux = davison_file['calving_obs']

ABUMIP GL FLUX

In [9]:
GL_flux_ABUMIP = xr.open_dataset(outputpath_GL + 'all_GL_fluxes.nc')

SMB

In [10]:
def start_end_year(mod, scen):
    if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
        ens_run = 'r1i1p1f2'
        to2300 = False
    elif mod in ['GISS-E2-1-H']:
        ens_run = 'r1i1p1f2'
        to2300 = True
    elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
        ens_run = 'r1i1p1f1'
        to2300 = True
    elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
        ens_run = 'r1i1p1f1'
        to2300 = False
    elif mod == 'UKESM1-0-LL':
        ens_run = 'r4i1p1f2'
        to2300 = True     
    elif mod == 'CESM2':
        ens_run = 'r11i1p1f1'
        to2300 = False        


    if scenario == 'historical':
        yystart = 1980 #1850
        yyend = 2014
    elif scenario == 'ssp245':
        yystart = 2015
        yyend = 2100  
    else:
        if to2300:
            yystart = 2015
            yyend = 2299
        else:
            yystart = 2015
            yyend = 2100  
    return ens_run, yystart, yyend

In [11]:
melt_atmo_list2 = []
inputpath_atmo_Chris = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_atmo_Nico = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/SMB_EMULATED/'
#for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
#            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
#           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']:
    
melt_atmo3_list = []
for scenario in ['historical','ssp126','ssp245','ssp585']:

    melt_atmo_list = []
    for mod in ['CESM2','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']: #'CNRM-CM6-1',
        
        _, yystart, yyend = start_end_year(mod, scenario)
        #print(mod,yystart,yyend)

        if os.path.exists(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc'):
            melt_atmo = xr.open_dataset(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc').sel(Nisf=rignot_isf)
            melt_atmo['time'] = melt_atmo['time'].dt.year
            melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
        else:
            melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc').sel(Nisf=rignot_isf)
            melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))

    for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 
               'MRI-ESM2-0']:

        _, yystart, yyend = start_end_year(mod, scenario)
        #print(mod,yystart,yyend)
        
        if mod == 'GFDL-CM4' and scenario == 'ssp126':
            melt_atmo = melt_atmo * np.nan
        
        else:
            melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc').sel(Nisf=rignot_isf)
            melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))

    melt_atmo_scenario = xr.concat(melt_atmo_list, dim='model')
    melt_atmo3_list.append(melt_atmo_scenario.assign_coords({'scenario': scenario}))
melt_atmo_all = xr.concat(melt_atmo3_list, dim='scenario').sel(Nisf=rignot_isf)
#melt_atmo_all = xr.concat(melt_atmo_list2, dim='model')


/home/burgardc/miniconda3/envs/py38/lib/python3.8/site-packages/xarray/coding/times.py:699: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/burgardc/miniconda3/envs/py38/lib/python3.8/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


OCEAN

In [14]:
Gt_list_models = []
box1_list_models = []

for mod in tqdm(['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']): #
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list1 = []

    for mparam in param_classic_list:

        melt1D_scenario_list = []
        for scenario in ['historical','ssp126','ssp245','ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoISMIP.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list1.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list1, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y']
    box1_classic = melt1D_classic['melt_1D_mean_myr_box1']
    
    melt1D_list2 = []
    for mparam in param_NN_list:

        melt1D_scenario_list = []
        for scenario in ['historical','ssp126','ssp245','ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoISMIP.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list2.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list2, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')
    box1_NN = melt1D_NN['predicted_melt'].sel(metrics='box1')
    
    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    box1_all = xr.concat([box1_classic, box1_NN], dim='param')
    
    Gt_list_models.append(Gt_all.assign_coords({'model':mod}))
    box1_list_models.append(box1_all.assign_coords({'model':mod}))

Gt_all_mod = xr.concat(Gt_list_models, dim='model')
box1_all_mod = xr.concat(box1_list_models, dim='model')


    
    

  0%|          | 0/14 [00:00<?, ?it/s]

ACCESS-ESM1-5
ACCESS-CM2
CESM2
CESM2-WACCM
CNRM-CM6-1
CNRM-ESM2-1
CanESM5
GFDL-CM4
GFDL-ESM4
GISS-E2-1-H
IPSL-CM6A-LR
MPI-ESM1-2-HR
MRI-ESM2-0
UKESM1-0-LL


ABUMIP GL FLUX

In [15]:
GL_flux_ABUMIP = xr.open_dataset(outputpath_GL + 'all_GL_fluxes.nc')
GL_flux_ref_varyingm = xr.open_dataset(outputpath_GL + 'all_GL_fluxes_varying_m.nc')

MASS BALANCE

In [16]:
mass_balance_simu = Gt_all_mod - melt_atmo_all + davison_file['calving_obs']

In [17]:
bay_weights = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison.nc')['bay_weights']

In [18]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

In [19]:
ensmean_weighted = uf.weighted_mean(mass_balance_simu, ['model','param'], bay_weights * sens_weights.sel(model=mass_balance_simu.model))
#ensmean_nonweighted = mass_balance_simu.mean(['model','param'])

In [20]:
da_weighted = mass_balance_simu.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model))

In [21]:
ensstat_weighted = da_weighted.quantile([0.05,0.5,0.95],dim=['model','param'])


In [22]:

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0

endyy_future = 2300

for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    #melt_obs_kisf = mass_balance_obs.sel(Nisf=kisf).values
    #melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    #ax[i].axhline(y=melt_obs_kisf,color='k')
    #ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    mass_bal_hist = ensstat_weighted.sel(scenario='historical')
    #mass_bal_ssp = ensstat_weighted.sel(scenario='ssp585')
    
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='blue', linewidth=2)
    
    ax[i].plot(np.arange(1990,2015), mass_bal_hist.sel(quantile=0.5,time=range(1990,2015),Nisf=kisf), color='black')
    ax[i].fill_between(x=np.arange(1990,2015), y1=mass_bal_hist.sel(quantile=0.05,time=range(1990,2015),Nisf=kisf), y2=mass_bal_hist.sel(quantile=0.95,time=range(1990,2015),Nisf=kisf), color='grey',alpha=0.2)
    ax[i].plot(np.arange(2015,endyy_future), ensstat_weighted.sel(quantile=0.5,time=range(2015,endyy_future),scenario='ssp126',Nisf=kisf), color='gold')
    ax[i].fill_between(x=np.arange(2015,endyy_future), y1=ensstat_weighted.sel(quantile=0.05,time=range(2015,endyy_future),scenario='ssp126',Nisf=kisf), y2=ensstat_weighted.sel(quantile=0.95,time=range(2015,endyy_future),scenario='ssp126',Nisf=kisf), color='gold',alpha=0.2)
    ax[i].plot(np.arange(2015,endyy_future), ensstat_weighted.sel(quantile=0.5,time=range(2015,endyy_future),scenario='ssp245',Nisf=kisf), color='darkorange')
    ax[i].fill_between(x=np.arange(2015,endyy_future), y1=ensstat_weighted.sel(quantile=0.05,time=range(2015,endyy_future),scenario='ssp245',Nisf=kisf), y2=ensstat_weighted.sel(quantile=0.95,time=range(2015,endyy_future),scenario='ssp245',Nisf=kisf), color='orange',alpha=0.2)
    ax[i].plot(np.arange(2015,endyy_future), ensstat_weighted.sel(quantile=0.5,time=range(2015,endyy_future),scenario='ssp585',Nisf=kisf), color='red')
    ax[i].fill_between(x=np.arange(2015,endyy_future), y1=ensstat_weighted.sel(quantile=0.05,time=range(2015,endyy_future),scenario='ssp585',Nisf=kisf), y2=ensstat_weighted.sel(quantile=0.95,time=range(2015,endyy_future),scenario='ssp585',Nisf=kisf), color='red',alpha=0.2)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path+'massbalance_future_ensmean_onlyweighted_ABUMIP_anoISMIP_all_scenarios.pdf')

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


  0%|          | 0/64 [00:00<?, ?it/s]

In [23]:

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0

endyy_future = 2300

for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    #melt_obs_kisf = mass_balance_obs.sel(Nisf=kisf).values
    #melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    #ax[i].axhline(y=melt_obs_kisf,color='k')
    #ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    mass_bal_hist = ensstat_weighted.sel(scenario='historical')
    #mass_bal_ssp = ensstat_weighted.sel(scenario='ssp585')
    
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='blue', linewidth=2)
    
    ax[i].plot(np.arange(1990,2015), mass_bal_hist.sel(quantile=0.5,time=range(1990,2015),Nisf=kisf), color='black')
    ax[i].fill_between(x=np.arange(1990,2015), y1=mass_bal_hist.sel(quantile=0.05,time=range(1990,2015),Nisf=kisf), y2=mass_bal_hist.sel(quantile=0.95,time=range(1990,2015),Nisf=kisf), color='grey',alpha=0.2)
    ax[i].plot(np.arange(2015,endyy_future), ensstat_weighted.sel(quantile=0.5,time=range(2015,endyy_future),scenario='ssp126',Nisf=kisf), color='gold')
    ax[i].fill_between(x=np.arange(2015,endyy_future), y1=ensstat_weighted.sel(quantile=0.05,time=range(2015,endyy_future),scenario='ssp126',Nisf=kisf), y2=ensstat_weighted.sel(quantile=0.95,time=range(2015,endyy_future),scenario='ssp126',Nisf=kisf), color='gold',alpha=0.2)
    ax[i].plot(np.arange(2015,endyy_future), ensstat_weighted.sel(quantile=0.5,time=range(2015,endyy_future),scenario='ssp245',Nisf=kisf), color='darkorange')
    ax[i].fill_between(x=np.arange(2015,endyy_future), y1=ensstat_weighted.sel(quantile=0.05,time=range(2015,endyy_future),scenario='ssp245',Nisf=kisf), y2=ensstat_weighted.sel(quantile=0.95,time=range(2015,endyy_future),scenario='ssp245',Nisf=kisf), color='orange',alpha=0.2)
    ax[i].plot(np.arange(2015,endyy_future), ensstat_weighted.sel(quantile=0.5,time=range(2015,endyy_future),scenario='ssp585',Nisf=kisf), color='red')
    ax[i].fill_between(x=np.arange(2015,endyy_future), y1=ensstat_weighted.sel(quantile=0.05,time=range(2015,endyy_future),scenario='ssp585',Nisf=kisf), y2=ensstat_weighted.sel(quantile=0.95,time=range(2015,endyy_future),scenario='ssp585',Nisf=kisf), color='red',alpha=0.2)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    ax[i].set_ylim(0,GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf))
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path+'massbalance_future_ensmean_onlyweighted_ABUMIP_anoISMIP_all_scenarios_ylimABUMIP.pdf')

  0%|          | 0/64 [00:00<?, ?it/s]

SINGLE PARAMS

In [24]:
ensstat_weighted_paramdim = da_weighted.quantile([0.05,0.5,0.95],dim=['model'])


In [28]:
ensstat_weighted_paramdim.param

<xarray.DataArray 'param' (param: 6)>
array(['linear_local', 'quadratic_local', 'quadratic_local_locslope',
       'lazero19', 'boxes_4_pismyes_picopno', 'NNsmall'], dtype=object)
Coordinates:
  * param       (param) object 'linear_local' 'quadratic_local' ... 'NNsmall'
    metrics     <U2 'Gt'
    box_nb_tot  int64 5
    config      int64 2

In [32]:

mparam = 'NNsmall'

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0

endyy_future = 2300

for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    #melt_obs_kisf = mass_balance_obs.sel(Nisf=kisf).values
    #melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    #ax[i].axhline(y=melt_obs_kisf,color='k')
    #ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    mass_bal_hist = ensstat_weighted_paramdim.sel(scenario='historical').sel(param=mparam)
    #mass_bal_ssp = ensstat_weighted.sel(scenario='ssp585')
    
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='blue', linewidth=2)
    
    ax[i].plot(np.arange(1990,2015), mass_bal_hist.sel(quantile=0.5,time=range(1990,2015),Nisf=kisf), color='black')
    ax[i].fill_between(x=np.arange(1990,2015), y1=mass_bal_hist.sel(quantile=0.05,time=range(1990,2015),Nisf=kisf), y2=mass_bal_hist.sel(quantile=0.95,time=range(1990,2015),Nisf=kisf), color='grey',alpha=0.2)
    ax[i].plot(np.arange(2015,endyy_future), ensstat_weighted_paramdim.sel(quantile=0.5,time=range(2015,endyy_future),scenario='ssp126',Nisf=kisf,param=mparam), color='gold')
    ax[i].fill_between(x=np.arange(2015,endyy_future), y1=ensstat_weighted_paramdim.sel(quantile=0.05,time=range(2015,endyy_future),scenario='ssp126',Nisf=kisf,param=mparam), y2=ensstat_weighted_paramdim.sel(quantile=0.95,time=range(2015,endyy_future),scenario='ssp126',Nisf=kisf,param=mparam), color='gold',alpha=0.2)
    ax[i].plot(np.arange(2015,endyy_future), ensstat_weighted_paramdim.sel(quantile=0.5,time=range(2015,endyy_future),scenario='ssp245',Nisf=kisf,param=mparam), color='darkorange')
    ax[i].fill_between(x=np.arange(2015,endyy_future), y1=ensstat_weighted_paramdim.sel(quantile=0.05,time=range(2015,endyy_future),scenario='ssp245',Nisf=kisf,param=mparam), y2=ensstat_weighted_paramdim.sel(quantile=0.95,time=range(2015,endyy_future),scenario='ssp245',Nisf=kisf,param=mparam), color='orange',alpha=0.2)
    ax[i].plot(np.arange(2015,endyy_future), ensstat_weighted_paramdim.sel(quantile=0.5,time=range(2015,endyy_future),scenario='ssp585',Nisf=kisf,param=mparam), color='red')
    ax[i].fill_between(x=np.arange(2015,endyy_future), y1=ensstat_weighted_paramdim.sel(quantile=0.05,time=range(2015,endyy_future),scenario='ssp585',Nisf=kisf,param=mparam), y2=ensstat_weighted_paramdim.sel(quantile=0.95,time=range(2015,endyy_future),scenario='ssp585',Nisf=kisf,param=mparam), color='red',alpha=0.2)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_ylim(0,GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf))
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path+'massbalance_future_ensmean_onlyweighted_ABUMIP_anoISMIP_all_scenarios'+mparam+'.pdf')

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
bay_weights

In [ ]:

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    #melt_obs_kisf = mass_balance_obs.sel(Nisf=kisf).values
    #melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    #ax[i].axhline(y=melt_obs_kisf,color='k')
    #ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    mass_bal_hist = mass_balance_simu.sel(scenario='historical')
    mass_bal_ssp = mass_balance_simu
    
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='blue', linewidth=2)
    
    for mmod in mass_bal_hist.model:
        for mparam in bay_weights.param:
            wweight = (bay_weights * sens_weights).sel(param=mparam,Nisf=kisf,model=mmod).values.item() *10
            ax[i].plot(np.arange(1990,2015), mass_bal_hist.sel(time=range(1990,2015),Nisf=kisf,param=mparam,model=mmod), color='black',alpha=wweight)
            ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(time=range(2015,2101),scenario='ssp126',Nisf=kisf,param=mparam,model=mmod), color='gold',alpha=wweight)
            ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(time=range(2015,2101),scenario='ssp245',Nisf=kisf,param=mparam,model=mmod), color='darkorange',alpha=wweight)
            ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(time=range(2015,2101),scenario='ssp585',Nisf=kisf,param=mparam,model=mmod), color='red',alpha=wweight)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#f.savefig(plot_path+'massbalance_future_ensmean_onlyweighted_ABUMIP_anoISMIP_all_scenarios.pdf')

In [ ]:
melt_weighted = Gt_all_mod.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model)).quantile([0.05,0.5,0.95],dim=['model','param'])

In [ ]:

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    #melt_obs_kisf = mass_balance_obs.sel(Nisf=kisf).values
    #melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    #ax[i].axhline(y=melt_obs_kisf,color='k')
    #ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    mass_bal_hist = Gt_all_mod.sel(scenario='historical')
    mass_bal_ssp = Gt_all_mod
    
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='blue', linewidth=2)
    
    
    for k,mparam in enumerate(bay_weights.param):
        if i == 0:
            ccolor = colors[k]
            print(mparam.values,ccolor)
        for mmod in mass_bal_hist.model:
            wweight = (bay_weights * sens_weights).sel(param=mparam,Nisf=kisf,model=mmod).values.item() * 30
            ax[i].plot(np.arange(1990,2015), mass_bal_hist.sel(time=range(1990,2015),Nisf=kisf,param=mparam,model=mmod),color = ccolor,alpha=wweight)
            ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(time=range(2015,2101),scenario='ssp126',Nisf=kisf,param=mparam,model=mmod),color = ccolor,alpha=wweight)
            ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(time=range(2015,2101),scenario='ssp245',Nisf=kisf,param=mparam,model=mmod),color = ccolor,alpha=wweight)
            ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(time=range(2015,2101),scenario='ssp585',Nisf=kisf,param=mparam,model=mmod),color = ccolor,alpha=wweight)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path+'basal_melt_future_colors_param.pdf')

In [ ]:

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    #melt_obs_kisf = mass_balance_obs.sel(Nisf=kisf).values
    #melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    #ax[i].axhline(y=melt_obs_kisf,color='k')
    #ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    mass_bal_hist = Gt_all_mod.sel(scenario='historical')
    mass_bal_ssp = Gt_all_mod
    
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='blue', linewidth=2)
    
    for mmod in mass_bal_hist.model.isel(model=[0]):
        for k,mparam in enumerate(bay_weights.param):
            ccolor = colors[k]
            wweight = (bay_weights * sens_weights).sel(param=mparam,Nisf=kisf,model=mmod).values.item() *10
            ax[i].plot(np.arange(1990,2015), mass_bal_hist.sel(time=range(1990,2015),Nisf=kisf,param=mparam,model=mmod),color = ccolor,alpha=wweight)
            ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(time=range(2015,2101),scenario='ssp126',Nisf=kisf,param=mparam,model=mmod),color = ccolor)
            ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(time=range(2015,2101),scenario='ssp245',Nisf=kisf,param=mparam,model=mmod),color = ccolor)
            ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(time=range(2015,2101),scenario='ssp585',Nisf=kisf,param=mparam,model=mmod),color = ccolor)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path+'basal_melt_future_colors_param_withlegend.pdf')